<a href="https://colab.research.google.com/github/ewotawa/secure_private_ai/blob/master/Section_3_Securing_Federated_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Section: Securing Federated Learning

- Lesson 1: Trusted Aggregator
- Lesson 2: Intro to Additive Secret Sharing
- Lesson 3: Intro to Fixed Precision Encoding
- Lesson 4: Secret Sharing + Fixed Precision in PySyft
- Final Project: Federated Learning wtih Encrypted Gradient Aggregation

# Lesson: Federated Learning with a Trusted Aggregator

In the last section, we learned how to train a model on a distributed dataset using Federated Learning. In particular, the last project aggregated gradients directly from one data owner to another. 

However, while in some cases it could be ideal to do this, what would be even better is to be able to choose a neutral third party to perform the aggregation.

As it turns out, we can use the same tools we used previously to accomplish this.

### Install libraries and dependencies

In [1]:
# PySyft

!pip install syft

import syft as sy

# PyTorch

!pip install torch
!pip install torchvision

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

import torchvision
from torchvision import datasets, transforms

# Numpy

import numpy as np

# time

import time

W0726 00:21:09.489979 140468923750272 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0726 00:21:09.510081 140468923750272 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



# Project: Federated Learning with a Trusted Aggregator

In [0]:
# try this project here!

### Instructor's Work

In [0]:
import syft as sy
import torch
hook = sy.TorchHook(torch)
from torch import nn, optim

In [0]:
# create a few workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")

In [5]:
# inform each worker that the other workers exist

bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([bob, alice])

W0726 00:21:16.155555 140468923750272 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0726 00:21:16.158713 140468923750272 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0726 00:21:16.160677 140468923750272 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0726 00:21:16.162790 140468923750272 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0726 00:21:16.164652 140468923750272 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0726 00:21:16.166789 140468923750272 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior


<VirtualWorker id:secure_worker #objects:0>

In [0]:
# Same Toy Dataset and simple linear model

data = torch.tensor([[0,0], [0,1], [1,0], [1,1.]], requires_grad=True)
target = torch.tensor([[0], [0], [1], [1.]], requires_grad=True)

In [0]:
# get pointers to training data on each worker by sending some training data to bob and alice

bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

alices_data = data[2:].send(alice)
alices_target = target[2:].send(alice)

In [0]:
# initialize a Toy Model

model = nn.Linear(2, 1)

In [0]:
# instead of having one model, have two different models that we send to the two different workers so that they can be averaged.

bobs_model = model.copy().send(bob)
alices_model = model.copy().send(alice)

In [0]:
# create two separate optimizers

bobs_opt = optim.SGD(params=bobs_model.parameters(), lr=0.1)
alices_opt = optim.SGD(params=alices_model.parameters(), lr=0.1)

In [11]:
# train the models

for round_iter in range(10):
  
  bobs_model = model.copy().send(bob)
  alices_model = model.copy().send(alice)
  
  bobs_opt = optim.SGD(params=bobs_model.parameters(), lr=0.1)
  alices_opt = optim.SGD(params=alices_model.parameters(), lr=0.1)
  
  for i in range(10):
    
    bobs_opt.zero_grad()
    bobs_pred = bobs_model(bobs_data)
    bobs_loss = ((bobs_pred - bobs_target) **2).sum()
    bobs_loss.backward()
    
    bobs_opt.step()
    bobs_loss = bobs_loss.get().data
    bobs_loss
    
    alices_opt.zero_grad()
    alices_pred = alices_model(alices_data)
    alices_loss = ((alices_pred - alices_target) **2).sum()
    alices_loss.backward()
    
    alices_opt.step()
    alices_loss = alices_loss.get().data
    alices_loss
  
  bobs_model.move(secure_worker)
  alices_model.move(secure_worker)
  
  with torch.no_grad():
    weights = model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
    bias = model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())
  
  secure_worker.clear_objects()
  
  print("Iteration:\t", round_iter)
  print("Bob:\t", str(bobs_loss), "\tAlice:\t", str(alices_loss))
  print("Weights: ", str(weights), "\nBias: ", str(bias),"\n")

Iteration:	 0
Bob:	 tensor(0.0728) 	Alice:	 tensor(0.0097)
Weights:  Parameter containing:
tensor([[-0.3562, -0.2346]], requires_grad=True) 
Bias:  Parameter containing:
tensor([0.6973], requires_grad=True) 

Iteration:	 1
Bob:	 tensor(0.0293) 	Alice:	 tensor(0.0002)
Weights:  Parameter containing:
tensor([[-0.1890, -0.1173]], requires_grad=True) 
Bias:  Parameter containing:
tensor([0.5831], requires_grad=True) 

Iteration:	 2
Bob:	 tensor(0.0151) 	Alice:	 tensor(0.0002)
Weights:  Parameter containing:
tensor([[-0.0401, -0.0689]], requires_grad=True) 
Bias:  Parameter containing:
tensor([0.4888], requires_grad=True) 

Iteration:	 3
Bob:	 tensor(0.0091) 	Alice:	 tensor(0.0005)
Weights:  Parameter containing:
tensor([[ 0.0935, -0.0457]], requires_grad=True) 
Bias:  Parameter containing:
tensor([0.4156], requires_grad=True) 

Iteration:	 4
Bob:	 tensor(0.0061) 	Alice:	 tensor(0.0006)
Weights:  Parameter containing:
tensor([[ 0.2118, -0.0335]], requires_grad=True) 
Bias:  Parameter contai

# Lesson: Intro to Additive Secret Sharing

While being able to have a trusted third party to perform the aggregation is certainly nice, in an ideal setting we wouldn't have to trust anyone at all. This is where Cryptography can provide an interesting alterantive. 

Specifically, we're going to be looking at a simple protocol for Secure Multi-Party Computation called Additive Secret Sharing. This protocol will allow multiple parties (of size 3 or more) to aggregate their gradients without the use of a trusted 3rd party to perform the aggregation. In other words, we can add 3 numbers together from 3 different people without anyone ever learning the inputs of any other actors.

Let's start by considering the number 5, which we'll put into a varible x

In [0]:
x = 5

Let's say we wanted to SHARE the ownership of this number between two people, Alice and Bob. We could split this number into two shares, 2, and 3, and give one to Alice and one to Bob

In [13]:
bob_x_share = 2
alice_x_share = 3

decrypted_x = bob_x_share + alice_x_share
decrypted_x

5

Note that neither Bob nor Alice know the value of x. They only know the value of their own SHARE of x. Thus, the true value of X is hidden (i.e., encrypted). 

The truly amazing thing, however, is that Alice and Bob can still compute using this value! They can perform arithmetic over the hidden value! Let's say Bob and Alice wanted to multiply this value by 2! If each of them multiplied their respective share by 2, then the hidden number between them is also multiplied! Check it out!

In [14]:
bob_x_share = 2 * 2
alice_x_share = 3 * 2

decrypted_x = bob_x_share + alice_x_share
decrypted_x

10

This even works for addition between two shared values!!

In [15]:
# encrypted "5"
bob_x_share = 2
alice_x_share = 3

# encrypted "7"
bob_y_share = 5
alice_y_share = 2

# encrypted 5 + 7
bob_z_share = bob_x_share + bob_y_share
alice_z_share = alice_x_share + alice_y_share

decrypted_z = bob_z_share + alice_z_share
decrypted_z

12

As you can see, we just added two numbers together while they were still encrypted!!!

One small tweak - notice that since all our numbers are positive, it's possible for each share to reveal a little bit of information about the hidden value, namely, it's always greater than the share. Thus, if Bob has a share "3" then he knows that the encrypted value is at least 3.

This would be quite bad, but can be solved through a simple fix. Decryption happens by summing all the shares together MODULUS some constant. I.e.

In [16]:
x = 5

Q = 23740629843760239486723

bob_x_share = 23552870267 # <- a random number
alice_x_share = Q - bob_x_share + x
alice_x_share

23740629843736686616461

In [17]:
(bob_x_share + alice_x_share) % Q

5

So now, as you can see, both shares are wildly larger than the number being shared, meaning that individual shares no longer leak this inforation. However, all the properties we discussed earlier still hold! (addition, encryption, decryption, etc.)

# Project: Build Methods for Encrypt, Decrypt, and Add 

In this project, you must take the lessons we learned in the last section and write general methods for encrypt, decrypt, and add. Store shares for a variable in a tuple like so.

In [0]:
x_share = (2,5,7)

Even though normally those shares would be distributed amongst several workers, you can store them in ordered tuples like this for now :)

In [0]:
# try this project here!

### My work

Create a method called <strong>encrypt()</strong>  
<ul>
  <li>Accepts two input parameters:
    <ul>
      <li>Number to be encrypted</li>
      <li>Number of shares to be split into</li>
    </ul>
  </li>
  <li>Returns a tuple of shares</li>
</ul>

Create a method called <strong>decrypt()</strong>  
* Accepts as input a tuple of shares 
* Returns the decrypted value 

Create a method called <strong>add()</strong> 
* Accepts two tuples of shares 
* Returns a single tuple of shares which are added correctly according to the additive secret sharing protocol.

In [0]:
def encrypt(orig_number, num_shares, Q):
  x = int(orig_number)
  
  # Q = 23740629843760239486723 #large prime number
  
  shares = []
  share_sum = int()
  
  for i in range(num_shares - 1):
    share = torch.randn(1) * 100000000000
    shares.append(int(share.item()))
  
  last_share = Q - int(sum(shares)) + x
  shares.append(last_share)
  
  sum_out = sum(shares)
  
  shares = tuple(shares)
  
  # validation:
  # print(sum_out % Q)
  
  return(shares)

In [0]:
def decrypt(encrypted_shares, Q):
  sum_shares = sum(encrypted_shares)
  decrypted = sum_shares % Q
  return decrypted

In [22]:
Q = 23740629843760239486723 #large prime number 
shares = encrypt(5, 200, Q)
print(shares)
decrypted = decrypt(shares, Q)
print(decrypted)

(48404017152, 37268344832, 45172924416, 214172139520, -171496161280, -36272275456, 1325095424, 13208155136, 45870018560, -195716169728, 20030484480, -21258268672, -54761136128, 35366637568, 92163555328, -88563277824, 194362376192, 836648576, -24129884160, 61459886080, 8927991808, -105625640960, 50138685440, 48976003072, 47181570048, -5782382592, -109374029824, 12370370560, -49431748608, 110995382272, 78522023936, -22257340416, -75291254784, 181980282880, 51208798208, -65850863616, 1362587776, 249025462272, 28094949376, 149882224640, -17737238528, 10544888832, -17314750464, -13725630464, 111559933952, 89403785216, 104940388352, -15262956544, 170423894016, -12734749696, 90204061696, 91082997760, -22575351808, 190532780032, 209165713408, 107776794624, 61423300608, 157157523456, -53264359424, 21686482944, 21910296576, -197568577536, -88690532352, -124373336064, -111401738240, 93403791360, 39465037824, 64173518848, -485526048, -177425350656, 155931852800, -25453158400, 72493432832, -3135059

In [0]:
def add(tuple1, tuple2):
  sum_tuple = tuple1 + tuple2
  return sum_tuple

In [24]:
tuple1 = encrypt(5, 5, Q)
tuple2 = encrypt(10, 5, Q)

sum_tuple = add(tuple1, tuple2)
print(sum_tuple)

decrypt(sum_tuple, Q)

(244839448576, -101526904832, 54522036224, -5230072832, 23740629843567634979592, -38770032640, -178999951360, 177767972864, -71160422400, 23740629843871401920269)


15

### Instructor's work

In [0]:
import random

In [0]:
Q = 23740629843760239486723 #large prime number 

In [0]:
def encrypt(x, n_shares):
  shares = list()
  for i in range(n_shares - 1):
    shares.append(random.randint(0, Q))
  final_share = Q - (sum(shares) % Q) + x
  shares.append(final_share)
  return tuple(shares)

In [28]:
encrypt(5, n_shares=10)

(19719178205133768498514,
 22437704552807729263033,
 8743514729836978573972,
 9994137450193379136389,
 12921063178089361404680,
 17681291228760577452375,
 15818014619598394287843,
 19596789109218081686548,
 20361092631404725080654,
 18911623201278681023058)

In [0]:
def decrypt(shares):
  return sum(shares) % Q

In [30]:
decrypt(encrypt(5, 10))

5

In [0]:
def add(a, b):
  c = list()
  assert(len(a) == len(b))
  for i in range(len(a)):
    c.append((a[i] + b[i]) % Q)
  return tuple(c)

In [32]:
decrypt(add(encrypt(5, 10), encrypt(5, 10)))

10

# Lesson: Intro to Fixed Precision Encoding

As you may remember, our goal is to aggregate gradients using this new Secret Sharing technique. However, the protocol we've just explored in the last section uses positive integers. However, our neural network weights are NOT integers. Instead, our weights are decimals (floating point numbers).

Not a huge deal! We just need to use a fixed precision encoding, which lets us do computation over decimal numbers using integers!

In [0]:
BASE=10
PRECISION=4

In [0]:
def encode(x):
    return int((x * (BASE ** PRECISION)) % Q)

def decode(x):
    return (x if x <= Q/2 else x - Q) / BASE**PRECISION

In [35]:
encode(3.5)

35000

In [36]:
decode(35000)

3.5

In [37]:
x = encrypt(encode(5.5), 3)
y = encrypt(encode(2.3), 3)
z = add(x,y)
decode(decrypt(z))

7.8

# Lesson: Secret Sharing + Fixed Precision in PySyft

While writing things from scratch is certainly educational, PySyft makes a great deal of this much easier for us through its abstractions.

In [0]:
bob = bob.clear_objects()
alice = alice.clear_objects()
secure_worker = secure_worker.clear_objects()

In [0]:
x = torch.tensor([1,2,3,4,5])

### Secret Sharing Using PySyft

We can share using the simple .share() method!

In [0]:
x = x.share(bob, alice, secure_worker)

In [41]:
bob._objects

{51351644448: tensor([1542841711977177559,  826536590925762557,  278092298435898122,
         3609678328164168715,  492195322488907427])}

and as you can see, Bob now has one of the shares of x! Furthermore, we can still call addition in this state, and PySyft will automatically perform the remote execution for us!

In [0]:
y = x + x

In [43]:
y

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:15186991494 -> bob:66313699950]
	-> (Wrapper)>[PointerTensor | me:36565099520 -> alice:77462307685]
	-> (Wrapper)>[PointerTensor | me:59613647163 -> secure_worker:69329711773]
	*crypto provider: me*

In [44]:
y.get()

tensor([ 2,  4,  6,  8, 10])

### Fixed Precision using PySyft

We can also convert a tensor to fixed precision using .fix_precision()

In [0]:
x = torch.tensor([0.1,0.2,0.3])

In [46]:
x

tensor([0.1000, 0.2000, 0.3000])

In [0]:
x = x.fix_prec()

In [48]:
x.child.child

tensor([100, 200, 300])

In [0]:
y = x + x

In [50]:
y = y.float_prec()
y

tensor([0.2000, 0.4000, 0.6000])

### Shared Fixed Precision

And of course, we can combine the two!

In [51]:
x = torch.tensor([0.1, 0.2, 0.3])
print(x)

tensor([0.1000, 0.2000, 0.3000])


In [52]:
x = x.fix_prec().share(bob, alice, secure_worker)
print(x)

(Wrapper)>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:10930538663 -> bob:67871063829]
	-> (Wrapper)>[PointerTensor | me:36219304287 -> alice:47764675786]
	-> (Wrapper)>[PointerTensor | me:67734829532 -> secure_worker:52285622123]
	*crypto provider: me*


In [53]:
y = x + x
print(x)

(Wrapper)>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:10930538663 -> bob:67871063829]
	-> (Wrapper)>[PointerTensor | me:36219304287 -> alice:47764675786]
	-> (Wrapper)>[PointerTensor | me:67734829532 -> secure_worker:52285622123]
	*crypto provider: me*


In [54]:
y.get().float_prec()


tensor([0.2000, 0.4000, 0.6000])

Make sure to make the point that people can see the model averages in the clear.

# Final Project: Federated Learning with Encrypted Gradient Aggregation

Build on the first project, where you perform federated learning with a trusted, secure aggregator.

<ul>
  <li>Take the same neural network you used in the first project. </li>
  <li>Aggregate gradients using additive secret sharing and fixed-precision encoding.  </li>
  <li>Use at least three data owners per aggregation.  
    <ul>
      <li>Ensures that no one will ever see anyone's gradients other than their own. Protects privacy without needing to trust a secure aggregator.  </li>
    </ul>
  </li>
</ul>  

My approach:
<ul>
<li>Work with the approach outlined in the <a href="https://github.com/OpenMined/PySyft/blob/dev/examples/tutorials/Part%2011%20-%20Secure%20Deep%20Learning%20Classification.ipynb">OpenMined notebook</a> exercise.</li>
  <li>Use the same dummy data approach as final project for Section 2.</li>
</ul>

In [0]:
# create a few workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")

In [0]:
# Same Toy Dataset and simple linear model

data = torch.tensor([[0,0], [0,1], [1,0], [1,1.]], requires_grad=True)
target = torch.tensor([[0], [0], [1], [1.]], requires_grad=True)

In [0]:
# get pointers to training data on each worker by sending some training data to bob and alice

bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

alices_data = data[2:].send(alice)
alices_target = target[2:].send(alice)

In [0]:
# initialize a Toy Model

model = nn.Linear(2, 1)

In [0]:
# instead of having one model, have two different models that we send to the two different workers so that they can be averaged.

bobs_model = model.copy().send(bob)
alices_model = model.copy().send(alice)

In [0]:
# create two separate optimizers

bobs_opt = optim.SGD(params=bobs_model.parameters(), lr=0.1)
alices_opt = optim.SGD(params=alices_model.parameters(), lr=0.1)

In [61]:
# train the models

for round_iter in range(10):
  
  bobs_model = model.copy().send(bob)
  alices_model = model.copy().send(alice)
  
  bobs_opt = optim.SGD(params=bobs_model.parameters(), lr=0.1)
  alices_opt = optim.SGD(params=alices_model.parameters(), lr=0.1)
  
  for i in range(10):
    
    bobs_opt.zero_grad()
    bobs_pred = bobs_model(bobs_data)
    bobs_loss = ((bobs_pred - bobs_target) **2).sum()
    bobs_loss.backward()
    
    bobs_opt.step()
    bobs_loss = bobs_loss.get().data
    bobs_loss
    
    alices_opt.zero_grad()
    alices_pred = alices_model(alices_data)
    alices_loss = ((alices_pred - alices_target) **2).sum()
    alices_loss.backward()
    
    alices_opt.step()
    alices_loss = alices_loss.get().data
    alices_loss
  
  bobs_model.move(secure_worker)
  alices_model.move(secure_worker)
  
  # weights
  bob_w = bobs_model.weight.data.child.clone().get()
  alice_w = alices_model.weight.data.child.clone().get()
  
  bob_w_s = bob_w.fix_prec().share(bob, alice, secure_worker)
  alice_w_s = alice_w.fix_prec().share(bob, alice, secure_worker)
  
  w_avg_get_fl= ((bob_w_s[0] + alice_w_s[0]) / 2).get().float_prec()
  w_l = w_avg_get_fl.tolist()
  w_ll = list()
  w_ll.append(w_l)
  
  w = torch.tensor(w_ll, requires_grad=True)
  
  # bias
  bob_b = bobs_model.bias.data.child.clone().get()
  alice_b = alices_model.bias.data.child.clone().get()
  
  bob_b_s = bob_b.fix_prec().share(bob, alice, secure_worker)
  alice_b_s = alice_b.fix_prec().share(bob, alice, secure_worker)
  
  b_avg_get_fl= ((bob_b_s[0] + alice_b_s[0]) / 2).get().float_prec()
  b_l = b_avg_get_fl.tolist()
  b_ll = list()
  b_ll.append(b_l)
  
  b = torch.tensor(b_ll, requires_grad=True)
  
  with torch.no_grad():
    weights = model.weight.set_(w)
    bias = model.bias.set_(b)
  
  secure_worker.clear_objects()
  
  print("Iteration:\t", round_iter)
  print("Bob:\t", str(bobs_loss), "\tAlice:\t", str(alices_loss))
  print("Weights: ", str(weights), "\nBias: ", str(bias),"\n")

Iteration:	 0
Bob:	 tensor(0.0090) 	Alice:	 tensor(0.0316)
Weights:  Parameter containing:
tensor([[0.3870, 0.1730]], requires_grad=True) 
Bias:  Parameter containing:
tensor([[0.1740]], requires_grad=True) 

Iteration:	 1
Bob:	 tensor(0.0003) 	Alice:	 tensor(0.0064)
Weights:  Parameter containing:
tensor([[0.4820, 0.0610]], requires_grad=True) 
Bias:  Parameter containing:
tensor([[0.1750]], requires_grad=True) 

Iteration:	 2
Bob:	 tensor(0.0001) 	Alice:	 tensor(0.0018)
Weights:  Parameter containing:
tensor([[0.5600, 0.0200]], requires_grad=True) 
Bias:  Parameter containing:
tensor([[0.1700]], requires_grad=True) 

Iteration:	 3
Bob:	 tensor(0.0005) 	Alice:	 tensor(0.0007)
Weights:  Parameter containing:
tensor([[0.6230, 0.0030]], requires_grad=True) 
Bias:  Parameter containing:
tensor([[0.1560]], requires_grad=True) 

Iteration:	 4
Bob:	 tensor(0.0006) 	Alice:	 tensor(0.0003)
Weights:  Parameter containing:
tensor([[ 0.6740, -0.0040]], requires_grad=True) 
Bias:  Parameter contai